In [ ]:
from IPython.display import Markdown, display

In [ ]:
def printmd(string):
    display(Markdown(string))

## Motion correction

In [ ]:
print("Motion correction has been performed using MotionCor2.\n")

print("Please cite:")
printmd("1. Zheng, S., Palovcak, E., Armache, JP. et al. "
        "MotionCor2: anisotropic correction of beam-induced motion for improved cryo-electron microscopy. "
        "*Nat Methods* **14**, 331–332 (2017).")